In [1]:
import geemap
import ee
import numpy as np
import matplotlib.pyplot as plt

In [3]:
ee.Authenticate()
ee.Initialize()

In [4]:
geemap.update_package()

Unzipping geemap-master.zip ...
Data downloaded to: C:\Users\nazin\Downloads\geemap-master

Please comment out 'geemap.update_package()' and restart the kernel to take effect:
Jupyter menu -> Kernel -> Restart & Clear Output


In [13]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [15]:
#basemaps = geemap.ee_basemaps.keys()
#print(basemaps)

In [7]:
##for basemap in basemaps:
   ## print(basemap)

In [5]:
##Map.split_map(left_layer = 'NLCD 2016 CONUS Land Cover' , right_layer ='NLCD 2001 CONUS Land Cover')
#Map

In [6]:
##geemap.ee_search()

In [7]:
##geemap.upload()

Loading a Shapefile

In [30]:
Shapefile_path = "D:/Land Use Land Cover Change/Study Sites Shapefile/Study_Sites_V1.shp"
Wards = geemap.shp_to_ee(Shapefile_path)
#roi = Wards.filter(ee.Filter.eq('WardNm2020', 'LUUBWE'))
#Map.addLayer(Wards, {}, 'Shapefile')
#print(Wards)
#plt.show()
Wards

No module named 'pycrs'


Image Collection

In [16]:
LUUBWE_sentinel = ee.ImageCollection('COPERNICUS/S2') \
.filterDate('2018-01-01', '2019-01-01') \
.filterBounds(Wards) \
.sort ('CLOUD_COVER') \
.first() \
.select('B[1-7]')

NameError: name 'Wards' is not defined

In [71]:
##LUUBWE_sentinel.get('CLOUD_COVER').getInfo()

Shapefile Collection

In [72]:
##Wards = ee.FeatureCollection('Shapfile_Path');
##roi = Wards.filter(ee.Filter.eq('WardNm2020','LUUBWE'));
##LUUBWE_boundary = Wards.filter(ee.Filter.eq('WardNm2020','LUUBWE'));
#roi = ee.FeatureCollection(LUUBWE_boundary).geometry();

Cliping the Image and adding it to the Map 

In [74]:
rgbVis = {'bands':['B4','B3','B2'], min: 0, max: 0.3};
##LUUBWE_sentinel_median = LUUBWE_sentinel.median();
##LUUBWE_sentinel_clip = LUUBWE_sentinel_median.clip(roi);
##LUUBWE_sentinel_clip = LUUBWE_sentinel.clip(roi);
##Map.addLayer(LUUBWE_boundary, {color:'red'});
##Map.addLayer(LUUBWE_boundary);
##Map.addLayer(LUUBWE_sentinel_clip, rgbVis);
##Map.addLayer(LUUBWE_sentinel.clip(Wards), rgbVis);
Map.centerObject(LUUBWE_sentinel, 10);

EEException: Image.select: Parameter 'input' is required.

Area of the shapefile

In [75]:
LUUBWE_Area =  LUUBWE_boundary.geometry().area();
LUUBWE_Area_sqKm = ee.Number(LUUBWE_Area).divide(1e6);
print( LUUBWE_Area_sqKm);

NameError: name 'LUUBWE_boundary' is not defined

Supervised classification

Merge all the training points

In [ ]:
var lable = "Class";
var sample = Waterbody.merge(Builtup).merge(Cropland).merge(Vegetation).merge(Bareland);
var bands = ['B2','B3','B4','B5','B6','B7'];
var training = LUUBWE_sentinel_clip.select(bands).sampleRegions({
  collection: sample, 
  properties: ['Class'], 
  scale: 30
});
print('training', training);

Training dataset divided in two ways
Traing the model: 80%
Test the model: 20% data for testing

In [ ]:
var dataset = training.randomColumn();
var trainset = dataset.filter(ee.Filter.lessThan('random', 0.8));
var testset = dataset.filter(ee.Filter.greaterThanOrEquals('random', 0.8));


Create the Model CART

In [ ]:
var classifier = ee.Classifier.smileCart().train(trainset, lable, bands);

Classify the sentinel image

In [ ]:
var classified = LUUBWE_sentinel_clip.classify(classifier);
print('classified', classified);

Visualisation of the classification

In [ ]:
var landcoverpalette = [
  'blue',
  'red',
  'yellow',
  'green',
  'grey',
  ];
  
Map.addLayer(classified.clip(roi),{'min': 0, 'max': 4, 'palette': landcoverpalette});

Get information about the trained classifier.

In [ ]:
print('Results of trained classifier', classifier.explain());

Acuracy assement

Get a confusion matrix and overall accuracy for the training sample.

In [ ]:
var trainAccuracy = classifier.confusionMatrix();
print('Training error matrix', trainAccuracy);
print('Training overall accuracy', trainAccuracy.accuracy());

Get a confusion matrix and overall accuracy for the validation sample.

In [ ]:
testset = trainset.classify(classifier);
var validationAccuracy = testset.errorMatrix(lable, 'classification');
print('Validation error matrix', validationAccuracy);
print('Validation accuracy', validationAccuracy.accuracy());

Calculating Area by Class by Reagion
Calculating the Nested Group area of classes

In [ ]:
var areaImage = ee.Image.pixelArea().addBands(classified);
var areas = areaImage.reduceRegion({
  reducer: ee.Reducer.sum().group({
    groupField:1,
    groupName: 'class'}), 
    geometry: roi,
    scale: 10,
    maxPixels: 1e13});

print(areaImage);

Flatening the Nested Gropus areas of classes

In [ ]:
var classAreas = ee.List(areas.get('groups'));
var classAreaLists = classAreas.map(function(item){
  var areaDic = ee.Dictionary(item)
  var classNumber = ee.Number(areaDic.get('class')).format()
  var area = ee.Number(areaDic.get('sum')).divide(1e6).round()
  return ee.List([classNumber, area])
});

var results = ee.Dictionary(classAreaLists.flatten());
print(results);

Export Image

In [ ]:
Export.image.toDrive({
image:classified,
description: 'LUUBWE_sentinel_classified_2019',
fileFormat: 'GeoTIFF',
scale: 10,
region: roi,
});

Export table to your google drive

In [ ]:
Export.table.toDrive({
        collection: results,
        description: 'classAreaLists', 
        folder: 'EE',
    fileNamePrefix: 'classAreas', 
    fileFormat: 'CSV'
    });